### Pre-processing data for histogram

In [1]:
shp = '/home/jakim/autocls/pureobj.shp'

lulc_col = 'idclcplus'

idxs_folder = '/mnt/d/autocls/sat_s2_cmb18'

ofolder = '/home/jakim/autocls/rsthisto'

In [2]:
import pandas as pd
import os

from glass.rd.shp import shp_to_obj
from glass.pys.oss import lst_folders_subfiles, lst_ff, fprop, mkdir
from glass.pys.tm import now_as_str
from glass.rst.confmtx import bin_confmxt_fmrst
from glass.wt import obj_to_tbl
from glass.dtt.split import split_shp_by_attr
from glass.wenv.grs import run_grass

In [3]:
# List images folder
imgs = lst_folders_subfiles(idxs_folder, files_format='.tif')

ikeys = list(imgs.keys())
refrst = imgs[ikeys[0]][0]

In [4]:
# Create Workspace
ws = mkdir(os.path.join(os.path.dirname(shp), now_as_str()), overwrite=True)

In [5]:
# Split Shapefile
gpkg = os.path.join(ws, 'pureobj.gpkg')
shps = split_shp_by_attr(
    shp, lulc_col, ws, _format='.gpkg',
    outname='pureobj', valinname=True
)

In [6]:
# Create GRASS GIS Session
loc = 'loc_histo'

gs = run_grass(ws, location=loc, srs=refrst)

import grass.script.setup as gsetup
gsetup.init(gs, ws, loc, 'PERMANENT')

'/tmp/tmp4640p5kk'

In [7]:
from glass.it.shp import shp_to_grs
from glass.it.rst import rst_to_grs, grs_to_rst
from glass.dtt.torst import grsshp_to_grsrst
from glass.rst.alg import grsrstcalc

In [8]:
for c in shps:
    shp_to_grs(gpkg, lyrname=shps[c], asCMD=True)

    shps[c] = grsshp_to_grsrst(shps[c], 1, f"rst_{shps[c]}")

In [9]:
# Import all rasters
for day in imgs:
    for i in range(len(imgs[day])):
        imgs[day][i] = rst_to_grs(imgs[day][i])

In [10]:
# Filter IDX by class and export
for cls in shps:
    for day in imgs:
        for r in imgs[day]:
            frst = grsrstcalc(f"{shps[cls]} * {r}", f"c{str(cls)}_{r}")
            grs_to_rst(frst, os.path.join(
                ofolder, f'{frst}.tif'
            ), as_cmd=True, is_int=None)